In [ ]:
# %load boiler_plate.py
# reload modules when they change
%load_ext autoreload
%autoreload 2

# Alow plotting
%matplotlib inline

# Import modules higher in folder hierarchy
import os
import sys


def add_path(path):
    if path not in sys.path:
        sys.path.append(path)


In [ ]:
add_path('..')

In [ ]:
from pipeline.db import fetch, inner_join, filter_videos

# Load data
## Videos data

In [ ]:
dbname='ds-wizards'
user='wizard'
host='192.95.32.117'
password='GaG23jVxZhMnQaU53r8o'

VQUERY = "select post_id, url from videos where status='ok'"

vres = fetch(host, dbname, user, password, VQUERY)
vres = [(post_id.split("_")[1], url) for post_id, url in vres]

## Tags

In [ ]:
dbname='ds-content-tags'
user='ds-content-tags'
password='0fXjWl592vNf1gYvIw8w'
host='192.95.32.117'

TQUERY = "select id, tags from videos where tags is not NULL"
TAGS = "select tag_id, name, path from content_tags"

tres = fetch(host, dbname, user, password, TQUERY)

tags = { 
    tag_id: (name, path) for (tag_id, name, path) in fetch(
        host, dbname, user, password, TAGS) 
}

In [ ]:
import itertools
from collections import Counter

In [ ]:
c = Counter(itertools.chain(*[tags for (_, tags) in tres]))
c_min10 = sorted([(tags[k][0], v  )for k, v in c.items() if v > 10], key = lambda x: x[1], )

In [ ]:
Counter([len(path) for name, path in tags.values()])

In [ ]:
[(name, path) for name, path in tags.values() if len(path) == 2]

## Join videos with tags

In [ ]:
videos = list(inner_join(tres, vres))
filtered, t2i, i2t = filter_videos(videos, 10)
print("Found %d videos with %d unique tags" % (len(filtered), len(t2i)))

In [ ]:
len(filtered), len(t2i)

In [ ]:
len(videos)

In [ ]:
# Take only the most common class from the list
import math

c = Counter([min([t2i.get(t, math.inf) for t in tags]) 
             for (id, tags, url) in videos])

In [ ]:
# Check the new size of the classes
len(c)

# DbPedia

In [ ]:
import requests

from urllib.parse import quote
from xml.etree import ElementTree

In [ ]:
XMLSN = "{http://lookup.dbpedia.org/}"
DBPEDIA_QUERY = "http://lookup.dbpedia.org/api/search/KeywordSearch?&QueryString=%s"
LABEL_PATH = ".//{0}Label".format(XMLSN)
DESCRIPTION_PATH = "{0}Description".format(XMLSN)
RESULT = "{0}Result".format(XMLSN)

In [ ]:
import re

def keep(label):
    if "owl#Thing" in label:
        return False
    
    if "from" in label:
        return False
    
    if "births" in label:
        return False
    
    if "xmlns" in label:
        return False
    
    if re.search(r"american people of [a-z]* descent", label):
        return False
    
    return True

def normalise(label):
    label = label.lower()
    
    return label

In [ ]:
def dbpedia_labels(tag, only_first = True):
    response = requests.get(DBPEDIA_QUERY % quote(tag))
    tree = ElementTree.fromstring(response.content)

    # It there no resutls, return tag and empy list
    if not tree.findall(RESULT):
        raise NameError

    tree = tree[0] if only_first else tree

    # Fetch labels form xml
    labels = [label.text for label in tree.findall(LABEL_PATH)]

    # Clean labels
    labels = [normalise(label) for label in labels if keep(label)]

    # Remove duplicates
    labels = list(set(labels))

    # Get Description
    description = tree.find(DESCRIPTION_PATH).text
    description = description.strip() if description else tag
    
    return description, labels

# Explore DbPedia tags

In [ ]:
for tag_id, (name, path) in itertools.islice(tags.items(), 3):
    try:
        desc, labels =  dbpedia_labels(name)
        print("%s | %s | \n%s" % (tag_id, name, desc))
        for label in labels:
            print("\t %s" % label)
    except NameError:
        print("[Exception] Could not find tags for %s" % name)

In [ ]:
def make_t2l(tags, logging_step=100, only_first = True):
    t2l = {}
    for index, tag_id in enumerate(tags):
        name, path =  tags[tag_id]
        if index % logging_step == 0:
            print("[%d] Fetching info for %s object in dbpedia" % (index, name))
        
        try:
            t2l[tag_id] = dbpedia_labels(name, only_first)
        except NameError:
            print("Could not find label for tag %s" % name)
        except Exception as e:
            print("Exception for %s [%s]" % name, e.message)
        
    return t2l

In [ ]:
t2l = make_t2l(tags)

In [ ]:
labels_raw = itertools.chain(*[set(labels) for (tag_id, (desc, labels)) in t2l.items()])
labels_raw = [normalise(label) for label in labels_raw if keep(label)]
labels = Counter(labels_raw)

In [ ]:
labels_top100 = set([name for name, count in labels.most_common()[:100]])

In [ ]:
t2l_top100 = {}
for tag_id, (desc, labels) in t2l.items():
    labels = [label for label in labels if label in labels_top100]
    if labels:
        t2l_top100[tag_id] = labels

In [ ]:
# Get filterd and raw key
keys_top100 = set(t2l_top100).intersection(set(tags))
keys = set(t2l).intersection(set(tags))

In [ ]:
from collections import namedtuple

In [ ]:
Tag = namedtuple('Tag', ['id', 'name', 'description', 'labels'], verbose=True)

In [ ]:
joined_tags = {k: Tag(k, tags[k][0], t2l[k][0], t2l[k][1])  for k in keys }

In [ ]:
len(keys)

In [ ]:
len(joined_tags)

In [ ]:
for tag_id, tag in itertools.islice(joined_tags.items(), 3):
    print("%s | %s | \n%s" % (tag_id, tag.name, tag.description))
    for label in tag.labels:
        print("\t %s" % label)

# IO

In [ ]:
import pickle

## Save tags

In [ ]:
with open("/data/video/dbpedia_tags.pickle", "wb") as handle:
    pickle.dump(t2l, handle)

## Load tags

In [ ]:
with open("/data/video/dbpedia_tags.pickle", "rb") as handle:
    t2l = pickle.load(handle)

# Clustering

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer

# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

# load nltk's SnowballStemmer as variabled 'stemmer'
stemmer = SnowballStemmer("english")

def filer_tokens(tokens, filter_stopwords=True):
    filtered = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered.append(token)
    
    # remove stop words
    if filter_stopwords:
        filtered = [token for token in filtered if token not in stopwords]
            
    return filtered
    
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = filer_tokens(tokens)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = filer_tokens(tokens)
    return filtered_tokens

def tokenize_and_lem(text):
    tokens = [token.lemma_ for token in nlp(example)]
    filtered_tokens = filer_tokens(tokens)
    return filtered_tokens

In [ ]:
example = "The Log Cabin Republicans (LCR) is an organization that works within the Republican Party to advocate equal rights for all Americans, including gays and lesbians in the United States. The group's constituency supports the Republican Party and advocates for the rights of gay and lesbian Americans. LCR is the only Republican organization dedicated to representing the interests of LGBT Americans and their allies."

In [ ]:
print(example)

In [ ]:
" ".join(tokenize_only(example))

In [ ]:
" ".join(tokenize_and_lem(example))

# TF-IDF and document similarity

In [ ]:
import numpy as np

In [ ]:
# Cluster only on descriptions
descriptions = [tag.description for tag in joined_tags.values()]

# Cluster only on labels
labels = [" ".join(tag.labels) for tag in joined_tags.values()]

# Cluster on both
descriptions_and_labels = ["%s %s" % tup for tup in zip(descriptions, labels)]

# Names
names = [tag.name for tag in joined_tags.values()]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(descriptions)
print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

# K-means clustering

In [ ]:
from sklearn.cluster import KMeans
num_clusters = 50
km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

In [ ]:
clusters = km.labels_.tolist()

In [ ]:
from collections import Counter 
import numpy as np
import matplotlib.pyplot as plt


x_labels, values = zip(*Counter(clusters).items())

indexes = np.arange(len(x_labels))

plt.figure(figsize=(20, 5))
plt.bar(indexes, values, width)
plt.xticks(indexes, x_labels)
plt.show()

In [ ]:
data = { 'names': names, 'descriptions': descriptions, 'labels': labels}

frame = pd.DataFrame(data, index = [clusters] , columns = ['names', 'descriptions', 'labels'])

In [ ]:
frame.ix[0]

In [ ]:
def print_top(km, terms, frame, ntop_words = 15, n_labels = 15):
    print("Top terms per cluster:")
    #sort cluster centers by proximity to centroid
    order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

    for i in range(km.n_clusters):
        top_words = [terms[i] for i in order_centroids[i, :ntop_words]]
        sample_labels = frame.ix[i]['names'][:n_labels].values

        print("Cluster %d" % i)
        print('WORDS: %s' % ', '.join(top_words))
        print("LABEL NAMES: %s" % ", ".join(sample_labels))
        print() #add whitespace

In [471]:
print_top(km, terms, frame)

Top terms per cluster:
Cluster 0
WORDS: transport, vehicl, use, bear, wheel, object, skateboard, motor, surfac, blood, magnet, aircraft, mechan, design, parachut
LABEL NAMES: Transport, Motorcycles, donut, Cars, Miniature UAV, bikes, Vacuum cleaner, Bus, Police car, Toddler, Carpool, Handshake, Fertility awareness, Quantum mechanics, Iran

Cluster 1
WORDS: comic, marvel, charact, comic book, book, appear, marvel comic, superhero, fiction, fiction charact, publish, book publish, comic book publish, appear comic, creat
LABEL NAMES: Comic book, Steve Harvey, Vapor, Wonder Woman (TV series), Catwoman, Squirrel Girl, thor, Black Panther (comics), Quicksilver (comics), Logan, Thimble, Captain America, Captain Marvel (DC Comics), Iron Man (2008 film), Bane (comics)

Cluster 2
WORDS: state, unit state, unit, north, ocean, west, south, popul, border, atlant, east, region, locat, mountain, extens
LABEL NAMES: Oklahoma, Arkansas, Abortion in the United States, The Home Depot, Ocean, Tennessee, Sp

# Hierarchical document clustering

In [472]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

In [475]:
np.random.choice(linkage_matrix, 100)

(4516, 4)

In [473]:
fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=names);
plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout
plt.close()

# Latent Dirichlet Allocation